In [1]:
import os
import numpy as np
import pandas as pd
import gzip
from Bio import SeqIO

In [2]:
def fasta_to_dataframe_gz(fasta_gz_path):
    """
    Converts a gzip-compressed FASTA file into a pandas DataFrame.
    
    Parameters:
    fasta_gz_path (str): Path to the gzip-compressed FASTA file (.fa.gz).

    Returns:
    pd.DataFrame: DataFrame containing two columns: 'ID' and 'Sequence'.
    """
    # Open the gzip-compressed FASTA file
    with gzip.open(fasta_gz_path, "rt") as handle:
        # Parse the FASTA file
        records = SeqIO.parse(handle, "fasta")
        
        # Extract data
        data = [(record.id, str(record.seq)) for record in records]
    
    # Create DataFrame
    df = pd.DataFrame(data, columns=['ID', 'Sequence'])
    return df

In [3]:

def unique_sequences(df):
    """
    Extracts unique sequences from the 'Sequence' column of a DataFrame.

    Parameters:
    df (pd.DataFrame): The input DataFrame with a 'Sequence' column.

    Returns:
    pd.DataFrame: A DataFrame containing only unique values from the 'Sequence' column.
    """
    # Select the 'Sequence' column and drop duplicates
    unique_df = df[['Sequence']].drop_duplicates()

    return unique_df


In [4]:
def generate_concensous_and_datasets(df):
    sequences=  df['Sequence'].values
    length = len(sequences[0])
    seqs_df = pd.DataFrame(sequences)
    concensous = []
    for i in range(length):
        if i % 10 == 0:
            print(f"Checking position {i}/{length}")
        seqs_df = pd.DataFrame(sequences, columns=['Sequence'])
        seqs_df['Sequence'] = seqs_df['Sequence'].map(lambda x: x[i])

        if len(seqs_df['Sequence'].unique()) == 1:
            unique = seqs_df['Sequence'].unique()[0]
            concensous.append((i, unique))

    print(f"Concensous positions: {len(concensous)}")
    print(f"Variants: {length - len(concensous)}")

    concensous_string = ''

    # print the concensous positions
    positions = [x[0] for x in concensous]
    chars = [x[1] for x in concensous]

    for i in range(length):
        if i in positions:
            concensous_string += chars[positions.index(i)]
        else:
            concensous_string += '?'

    variable_positions = [i for i in range(length) if i not in positions]

    df_only_variables  = df.copy()
    df_only_variables['Sequence'] = df_only_variables['Sequence'].apply(lambda x: ''.join(x[i] for i in variable_positions))

    return concensous_string, df_only_variables




In [5]:
for data_type in ['long_reads', 'short_reads']: 
    # get all the files in the directory
    files = os.listdir('data/'+data_type)
    # keep only ones that are .fa.gz
    files = [file for file in files if file.endswith('.fa.gz')]
    # get the fasta files
    for file in files:
        name = file.split('.fa.gz')[0]
        df = fasta_to_dataframe_gz('data/'+data_type+'/'+file)
        len_df = len(df['Sequence'])
        print(f'number of sequences in {name}: {len_df}')
        
        df = unique_sequences(df)
        concensous_string, df_only_variables = generate_concensous_and_datasets(df)

        # save the variable_positions to csv file
        df_only_variables.to_csv(f'data/{data_type}/{name}.csv')

        # save oncensous string to text file
        with open(f'data/{data_type}/{name}_concensous.txt', "w") as text_file:
            text_file.write(concensous_string)




number of sequences in GRIA-CNS-RESUB.C0x1291.aligned.sorted.MinRQ998.reads.degenerate: 71434
Checking position 0/2829
Checking position 10/2829
Checking position 20/2829
Checking position 30/2829
Checking position 40/2829
Checking position 50/2829
Checking position 60/2829
Checking position 70/2829
Checking position 80/2829
Checking position 90/2829
Checking position 100/2829
Checking position 110/2829
Checking position 120/2829
Checking position 130/2829
Checking position 140/2829
Checking position 150/2829
Checking position 160/2829
Checking position 170/2829
Checking position 180/2829
Checking position 190/2829
Checking position 200/2829
Checking position 210/2829
Checking position 220/2829
Checking position 230/2829
Checking position 240/2829
Checking position 250/2829
Checking position 260/2829
Checking position 270/2829
Checking position 280/2829
Checking position 290/2829
Checking position 300/2829
Checking position 310/2829
Checking position 320/2829
Checking position 330/2829